In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use('ggplot')

import pandas as pd
pd.set_option('precision', 4)

from audition import Auditioner
import catwalk.db

### Setting up the Auditioner instance
Currently you need to specify the set of `model_group_id`s and `train_end_time`s you want to use manually, so here we're reading a few sets out of the database.

Additionally, you need to specify a name for the best distance table when creating the `Auditioner` and should ensure it doesn't already exist.

For simplicity, we'll just look at precision@300_abs here.

In [ ]:
conn = catwalk.db.connect()

In [ ]:
sel = """
SELECT model_group_id
FROM results.model_groups
WHERE model_config->>'label_definition' = 'any_serious_violation'
ORDER BY RANDOM()
;
"""

model_groups = list(pd.read_sql(sel,conn)['model_group_id'])

In [ ]:
sel = """
SELECT DISTINCT train_end_time
FROM results.models
WHERE model_group_id IN ({})
    AND EXTRACT(MONTH FROM train_end_time) IN (1,4,7,10)
    AND train_end_time >= '2012-01-01'
ORDER BY train_end_time
;
""".format(', '.join(map(str, model_groups)))

end_times = list(pd.read_sql(sel, conn)['train_end_time'])

In [ ]:
aud = Auditioner(
    db_engine = conn,
    model_group_ids = model_groups,
    train_end_times = end_times,
    initial_metric_filters = [{'metric': 'precision@', 'parameter': '300_abs', 'max_from_best': 1.0, 'threshold_value': 0.0}],
    models_table = 'models',
    distance_table = 'kr_test_dist'
)

### Plotting the best distance metric and groups over time
This is done with the `plot_model_groups` method and may take a minute to generate.

In [ ]:
aud.plot_model_groups()

### Applying thresholds to weed out bad models
Here we use the `update_metric_filters` to apply a set of filters to the model groups we're considering in order to elminate poorly performing ones. The model groups will be plotted again after updating the filters.

In [ ]:
aud.update_metric_filters(
    [{
        'metric': 'precision@',
        'parameter': '300_abs',
        'max_from_best': 0.2,
        'threshold_value': 0.0
    }]
)

Apply a round of filtering, starting with no threshold_value and a fairly wide margin on max_from_best

In [ ]:
# how many model groups are left after the first round of filtering?
len(aud.thresholded_model_group_ids)

That didn't thin things out too much, so let's get a bit more agressive with both parameters:

In [ ]:
aud.update_metric_filters([{
    'metric': 'precision@',
    'parameter': '300_abs',
    'max_from_best': 0.1,
    'threshold_value': 0.5
}])
len(aud.thresholded_model_group_ids)

That's starting to look better, but we can probably narrow even a bit more...

In [ ]:
aud.update_metric_filters([{
    'metric': 'precision@',
    'parameter': '300_abs',
    'max_from_best': 0.05,
    'threshold_value': 0.65
}])
len(aud.thresholded_model_group_ids)

This looks like a better set of prospective models to consider. Could potentially even back off a little bit, but certainly seems like we've cleared out most of the worst models.

### Applying selection rules and calculating regrets for the narrowed set of models
The goal of audition is to narrow a very large number of model groups to a small number of best candidates, ideally making use of the full time series of information. There are several ways one could consider doing so, using over-time averages of the metrics of interest, weighted averages to balance between metrics, the distance from best metrics, and balancing metric average values and stability. 

Audition formalizes this idea through "selection rules" that take in the data up to a given point in time, apply some rule to choose a model group, and evaluate the performance of that chosen model in the subsequent time window, the `regret`. You can register, evaluate, and update selection rules associated with the `Auditioner` object as shown below.

In [ ]:
seln_rules = [{
        'shared_parameters': [
            {'metric': 'precision@', 'parameter': '300_abs'}
        ],
        'selection_rules': [
            {'name': 'best_current_value'},
            {'name': 'best_average_value'},
            {'name': 'most_frequent_best_dist', 'dist_from_best_case': [0.01, 0.05, 0.1, 0.15]}
        ]
    },
    {
        'shared_parameters': [
            {'metric': 'precision@', 'parameter': '300_abs'}
        ],
        'selection_rules': [
            {'name': 'best_avg_recency_weight', 'curr_weight': [1.5, 2.0, 5.0], 'decay_type': ['linear']}
        ]
    },
    {
        'shared_parameters': [{}],
        'selection_rules': [{'name': 'random_model_group'}]
    }]

In [ ]:
aud.register_selection_rule_grid(seln_rules)

Finally, when you have a selection rule grid you're happy with, the `selection_rule_model_group_ids` parameter of the `Auditioner` will give you the model groups chosen by the selection rules in the grid when applied to the most recent end time for use in application:

In [ ]:
aud.selection_rule_model_group_ids